In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0,1,2,3"

import easydel as ed
import jax

jax.devices()

HELLO FROM EASYDEL


2025-12-05 17:17:23,962 - numexpr.utils - INFO - Note: detected 256 virtual cores but NumExpr set to maximum of 64, check "NUMEXPR_MAX_THREADS" environment variable.
2025-12-05 17:17:23,963 - numexpr.utils - INFO - Note: NumExpr detected 256 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 16.
2025-12-05 17:17:23,963 - numexpr.utils - INFO - NumExpr defaulting to 16 threads.
2025-12-05 17:17:24,836 - eray-executor - INFO - Not initializing jax.distributed because no distributed config was provided, and no cluster was detected.
2025-12-05 17:17:24,837 - eray-executor - INFO - No auto-discovered ray address found. Using ray.init('local').
2025-12-05 17:17:24,837 - eray-executor - INFO - ray.init(address='local', namespace='eray-executor', **{})
2025-12-05 17:17:29,319	INFO worker.py:1908 -- Started a local Ray instance. View the dashboard at http://127.0.0.1:8265 
INFO:2025-12-05 17:17:32,917:jax._src.xla_bridge:822: Unable to initialize backend 'tpu': INTERNAL: Failed

[CudaDevice(id=0), CudaDevice(id=1), CudaDevice(id=2), CudaDevice(id=3)]

In [2]:
checkpoint_dir = "/pub/hofmann-scratch/dvruette/gidd-easydel/gidd-unif-3b/gidd/orbax"
# checkpoint_dir = "/pub/hofmann-scratch/dvruette/gidd-easydel/gidd-unif-10b/gidd/orbax"
# checkpoint_dir = "/pub/hofmann-scratch/dvruette/gidd-easydel/gidd-mask-3b/gidd/orbax"

In [3]:
state_dict_path = os.path.join(checkpoint_dir, "..", "state_dict.pt")

if "gidd-unif-3b" in checkpoint_dir:
    num_layers = 19
    hidden_size = 3072
    num_attn_heads = 24
elif "gidd-mask-3b" in checkpoint_dir:
    num_layers = 18
    hidden_size = 3072
    num_attn_heads = 24
elif "gidd-unif-10b" in checkpoint_dir:
    num_layers = 34
    hidden_size = 4608
    num_attn_heads = 36
else:
    raise ValueError("Unknown checkpoint")

In [4]:
from gidd_easydel.loading import load_checkpoint

mesh, module, tokenizer, dtype = load_checkpoint(
    checkpoint_dir,
    num_layers=num_layers,
    hidden_size=hidden_size,
    num_attn_heads=num_attn_heads,
)

HELLO FROM ORBAX


2025-12-05 17:17:43,081 - absl - INFO - [thread=MainThread] Failed to get flag value for EXPERIMENTAL_ORBAX_USE_DISTRIBUTED_PROCESS_ID.
2025-12-05 17:17:43,082 - absl - INFO - [process=0][thread=MainThread] CheckpointManager init: checkpointers=None, item_names=None, item_handlers=None, handler_registry=None
2025-12-05 17:17:43,084 - absl - INFO - Initialized registry DefaultCheckpointHandlerRegistry({('metrics', <class 'orbax.checkpoint._src.handlers.json_checkpoint_handler.JsonSaveArgs'>): <orbax.checkpoint._src.handlers.json_checkpoint_handler.JsonCheckpointHandler object at 0x7f9b141c7090>, ('metrics', <class 'orbax.checkpoint._src.handlers.json_checkpoint_handler.JsonRestoreArgs'>): <orbax.checkpoint._src.handlers.json_checkpoint_handler.JsonCheckpointHandler object at 0x7f9b141c7090>}).
2025-12-05 17:17:43,085 - absl - INFO - orbax-checkpoint version: 0.11.25
2025-12-05 17:17:43,086 - absl - INFO - [process=0][thread=MainThread] Using barrier_sync_fn: <function get_barrier_sync_f

/pub/hofmann-scratch/dvruette/gidd-easydel/gidd-unif-3b/gidd/orbax


2025-12-05 17:17:44,956 - absl - INFO - [process=0] /jax/checkpoint/read/gbytes_per_sec: 2 Bytes/s (total gbytes: 5.5 GiB) (time elapsed: a second) (per-host)
2025-12-05 17:17:44,958 - absl - INFO - Finished restoring checkpoint in 1.86 seconds from /pub/hofmann-scratch/dvruette/gidd-easydel/gidd-unif-3b/gidd/orbax/22500.
2025-12-05 17:17:44,958 - absl - INFO - {'step': 22500, 'event_type': 'restore', 'directory': '/pub/hofmann-scratch/dvruette/gidd-easydel/gidd-unif-3b/gidd/orbax', 'checkpointer_start_time': 1764951463.0989778, 'checkpointer_duration_secs': 1.859926462173462, 'checkpoint_manager_start_time': 1764951463.0985126, 'checkpoint_manager_duration_secs': 1.8603935241699219}
2025-12-05 17:17:44,959 - absl - INFO - [process=0][thread=MainThread][wait_until_finished] No Save Finalize thread to wait for. Returning.
2025-12-05 17:17:44,959 - absl - INFO - Closing _NonBlockingMetadataStore(enable_write=True, _write_lock=<locked _thread.RLock object owner=140536020145984 count=1 at 

In [5]:
import torch
import jax.numpy as jnp

from easydel.utils.parameters_transformation import StateDictConverter

state_dict = StateDictConverter.easydel_to_torch(module=module, dtype=jnp.float32)

Converting GiddForDiffusionLM to torch:   0%|          | 0/269 [00:00<?, ?it/s]

In [6]:
for k, v in state_dict.items():
    # state_dict[k] = v.to(torch.bfloat16).cpu()
    state_dict[k] = v.cpu()

In [7]:
torch.save(state_dict, state_dict_path)

In [ ]:
# import importlib
# import gidd_easydel.model.modeling_gidd_hf
# import gidd_easydel.model.gidd_configuration_hf
# importlib.reload(gidd_easydel.model.modeling_gidd_hf)
# importlib.reload(gidd_easydel.model.gidd_configuration_hf)

# from gidd_easydel.model.modeling_gidd_hf import GiddForDiffusionLM
# from gidd_easydel.model.gidd_configuration_hf import GiddConfig

# config = GiddConfig(
#     num_hidden_layers=6,
#     hidden_size=512,
#     num_attention_heads=8,
#     intermediate_size=512 * 4,
#     attention_bias=True,
#     mlp_bias=True,
#     torch_dtype=torch.bfloat16,
# )

# device = torch.device("cuda")

# model = GiddForDiffusionLM(config).to(device)
# model.eval()

# input_ids = torch.randint(0, len(tokenizer), (1, 12), device=device)
# noise_mask = torch.ones((1, 12), dtype=torch.bool, device=device)
# noise_mask[:, :noise_mask.shape[1] // 2] = False

# attn_mask = attn_mask = (noise_mask[..., None] >= noise_mask[..., None, :])

# with torch.no_grad():
#     outputs = model(
#         input_ids=input_ids,
#         attention_mask=attn_mask,
#     )

In [ ]:
import torch

import importlib
import gidd_easydel.model.modeling_gidd_hf
import gidd_easydel.model.gidd_configuration_hf
importlib.reload(gidd_easydel.model.modeling_gidd_hf)
importlib.reload(gidd_easydel.model.gidd_configuration_hf)

from gidd_easydel.model.modeling_gidd_hf import GiddForDiffusionLM
from gidd_easydel.model.gidd_configuration_hf import GiddConfig

device = torch.device("cuda")

config = GiddConfig(
    num_hidden_layers=num_layers,
    hidden_size=hidden_size,
    num_attention_heads=num_attn_heads,
    intermediate_size=hidden_size * 4,
    max_position_embeddings=2048,
    resid_scale=4.0,
    init_scale = 0.4 / hidden_size ** 0.5,
    weight_scaling=1.0,
    head_scaling=512.0 / hidden_size,
    use_qk_norm=True,
    attention_bias=True,
    mlp_bias=True,
    torch_dtype=torch.bfloat16,
)

model = GiddForDiffusionLM(config)

In [ ]:
state_dict = torch.load(state_dict_path, map_location="cpu")
model.load_state_dict(state_dict)

<All keys matched successfully>

In [ ]:
model.eval()
model.to(device)

# input_ids = torch.randint(0, len(tokenizer), (1, 8), device=device)
input_ids = torch.arange(8, device=device).unsqueeze(0)
noise_mask = torch.ones((1, 8), dtype=torch.bool, device=device)
noise_mask[:, :noise_mask.shape[1] // 2] = False

attn_mask = attn_mask = (noise_mask[..., None] >= noise_mask[..., None, :])

with torch.no_grad():
    outputs = model(
        input_ids=input_ids,
        attention_mask=attn_mask,
    )

In [ ]:
outputs.logits[0, [0, 1, 2, 3, 4, 5, 6, 7], [0, 1, 2, 3, 4, 5, 6, 7]]

tensor([ 5.6875, 11.5000,  7.7188,  2.0312, 32.7500,  2.3750, 18.7500,  4.0625],
       device='cuda:0', dtype=torch.bfloat16)